In [53]:
!pip install Unidecode

In [0]:
import pandas as pd
import string
import nltk
import numpy as np
import re
import json
import unidecode
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import minmax_scale

In [56]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Load dataset from metadata file
file=open("/content/drive/My Drive/ML project'20/YelpZip/metadata")
data=file.readlines()
file.close()
for ind in range(len(data)):
    data[ind]=data[ind].split()

In [0]:
# Load reviews from reviewcontent file
file2=open("/content/drive/My Drive/ML project'20/YelpZip/reviewContent")
text=file2.readlines()
file2.close()
for i in range(len(text)):
    text[i]=text[i].split("\t")
text=[t[3] for t in text]

In [0]:
Dataset=pd.DataFrame(data,columns=["user_id",'product_id','rating','label','date'])
Dataset["review_text"]=text

In [0]:
pd.set_option('mode.chained_assignment', None)

In [0]:
Dataset['rating']=Dataset['rating'].astype('float')
Dataset['date']=pd.to_datetime(Dataset['date'])
Dataset['day']=Dataset['date'].dt.day_name()
Dataset['year']=Dataset.date.dt.year
Dataset['label']=Dataset.label.astype(int)

In [63]:
Y={}
for y in Dataset.year:
  if y in Y:
    Y[y]+=1
  else:
    Y[y]=0
Y

{2004: 2,
 2005: 426,
 2006: 2261,
 2007: 7535,
 2008: 16781,
 2009: 31907,
 2010: 54578,
 2011: 80212,
 2012: 97961,
 2013: 131225,
 2014: 180674,
 2015: 5024}

In [75]:
x=Dataset[Dataset.year== 2012]
x=x.append(Dataset[Dataset.year==2013])
x=x.append(Dataset[Dataset.year==2014])
train_data=x.sample(n=25000)

(409863, 8)


In [77]:
test_data=Dataset[Dataset.year==2015]
print(train_data.shape)
print(test_data.shape)

(25000, 8)
(5025, 8)


In [80]:
# X_train['upper_case_word_count']=X_train['review_text'].apply(lambda x:len([y for y in x.split() if y) )
#Train Dataset
#Review centric features
train_data['word_count']=train_data['review_text'].apply(lambda x: len(x.split()))
train_data['punctuation_count']=train_data['review_text'].apply(lambda x: len(''.join(c for c in x if c in string.punctuation)))
train_data['char_count']=train_data['review_text'].apply(lambda x: len(x))
train_data['title_count']=train_data['review_text'].apply(lambda x: len([word for word in x.split() if word.istitle()]))
print("-----------review_centric features extracted-------------/")
# user-centric features
train_data['user_id_no_of_review'] = train_data.groupby('user_id')['user_id'].transform('size')
train_data['user_id_ave_rating'] = train_data.groupby('user_id')['rating'].transform('mean')
train_data['user_id_ave_no_words'] = train_data.groupby('user_id')['word_count'].transform('mean')
train_data['user_id_max_review_a_day'] = train_data.groupby(['user_id','day'])['user_id'].transform('size')
print("-----------user_centric features extracted-------------/")
#Product centric feature
train_data['product_id_no_of_review']=train_data.groupby('product_id')['product_id'].transform('size')
train_data['product_id_ave_rating']=train_data.groupby('product_id')['rating'].transform('mean')
train_data['product_id_ave_no_of_words']=train_data.groupby('product_id')['word_count'].transform('mean')
train_data['product_id_max_review_a_day']=train_data.groupby(['product_id','day'])['user_id'].transform('size')
print("-----------product_centric features extracted-------------/\n\n")

#Test dataset
test_data['word_count']=test_data['review_text'].apply(lambda x: len(x.split()))
test_data['punctuation_count']=test_data['review_text'].apply(lambda x: len(''.join(c for c in x if c in string.punctuation)))
test_data['char_count']=test_data['review_text'].apply(lambda x: len(x))
test_data['title_count']=test_data['review_text'].apply(lambda x: len([word for word in x.split() if word.istitle()]))
print("-----------review_centric features extracted-------------/")
# user-centric features
test_data['user_id_no_of_review'] = test_data.groupby('user_id')['user_id'].transform('size')
test_data['user_id_ave_rating'] = test_data.groupby('user_id')['rating'].transform('mean')
test_data['user_id_ave_no_words'] = test_data.groupby('user_id')['word_count'].transform('mean')
test_data['user_id_max_review_a_day'] = test_data.groupby(['user_id','day'])['user_id'].transform('size')
print("-----------user_centric features extracted-------------/")
#Product centric feature
test_data['product_id_no_of_review']=test_data.groupby('product_id')['product_id'].transform('size')
test_data['product_id_ave_rating']=test_data.groupby('product_id')['rating'].transform('mean')
test_data['product_id_ave_no_of_words']=test_data.groupby('product_id')['word_count'].transform('mean')
test_data['product_id_max_review_a_day']=test_data.groupby(['product_id','day'])['user_id'].transform('size')
print("-----------product_centric features extracted-------------/")

-----------review_centric features extracted-------------/
-----------user_centric features extracted-------------/
-----------product_centric features extracted-------------/


-----------review_centric features extracted-------------/
-----------user_centric features extracted-------------/
-----------product_centric features extracted-------------/


In [81]:
train_data.head()

,user_id,product_id,rating,label,date,review_text,day,year,word_count,punctuation_count,char_count,title_count,user_id_no_of_review,user_id_ave_rating,user_id_ave_no_words,user_id_max_review_a_day,product_id_no_of_review,product_id_ave_rating,product_id_ave_no_of_words,product_id_max_review_a_day
112040,61337,1006,5.0,1,2013-10-24,"It's very rare for me to give a place 5 stars,...",Thursday,2013,244,35,1344,23,2,3.5,263.0,1,17,4.352941,161.176471,4
118283,84141,1050,4.0,1,2014-08-17,Two words: Pork Belly! I would come back just...,Sunday,2014,145,22,826,14,1,4.0,145.0,1,15,3.466667,150.200000,4
479516,226573,3964,4.0,1,2012-12-03,Went for a holiday meal with the family. Good ...,Monday,2012,14,4,86,2,1,4.0,14.0,1,12,3.166667,130.500000,3
103566,76102,2669,3.0,1,2014-09-10,Selection was mediocre. My steak was cooked p...,Wednesday,2014,102,15,611,10,1,3.0,102.0,1,4,3.500000,237.000000,1
100113,35307,914,4.0,1,2012-12-22,Had the lamb curry platter and my wife had the...,Saturday,2012,36,8,202,3,2,3.0,67.0,1,2,3.500000,44.000000,1


In [0]:
train_data.to_csv("/content/drive/My Drive/ML project'20/YelpZip/train.csv")
test_data.to_csv("/content/drive/My Drive/ML project'20/YelpZip/test.csv")